In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from notebookutils import mssparkutils
from datetime import datetime
from pyspark.sql.functions import explode, lit, col


In [ ]:
def readQuotaJsonFile(filePath, current_date_int):
    fileName = filePath.split('/')[-1]
    SubscriptionKey = int(fileName.split('_')[1])
    df = spark.read.option("multiline", "true").json(filePath)
    df = df.select(explode(df.responses).alias('col'))
    df= df.select("col.name", explode("col.content.value").alias("value"))
    df = df.select(col("name").alias("RegionId"), col("value.currentValue").cast("int").alias("Usage"), col("value.limit").cast("int").alias("Limit"))
    df = df.withColumn("DateKey", lit(current_date_int)).withColumn("SubscriptionKey", lit(SubscriptionKey))
    return df

In [ ]:
pathQuotas = "Files/Data/Quotas"
quotasListFiles = mssparkutils.fs.ls(pathQuotas)

In [ ]:
current_date = int(datetime.now().strftime("%Y%m%d"))

#Clean the date if exists in case of multiple run in the same day. We only keep 1 snapshot per day 
if spark.catalog.tableExists("Quota_fabric"):
     print("Table exists, snapshot will be clean.")
     spark.sql(f"DELETE FROM Quota_fabric WHERE DateKey = {str(current_date)}")

In [ ]:
for Quotafile in quotasListFiles:
    if Quotafile.isFile and Quotafile.name.endswith(".json"):
        print(f"integrate quota data from file {Quotafile.name}")
        df_quota = readQuotaJsonFile(Quotafile.path, current_date)
        df_quota.write.mode("append").saveAsTable("Quota_fabric")

print("Load Done")

#Clean raw files
for Quotafile in quotasListFiles:
    if Quotafile.isFile and Quotafile.name.endswith(".json"):
        mssparkutils.fs.rm(Quotafile.path)

print("Clean Done")